#### Import libraries / modules

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
os.getcwd()

'C:\\Users\\p.schambach\\Desktop\\DSR\\drone_steering\\models\\playground'

In [5]:
os.chdir('../..')

In [6]:
from app_local.module import DataEnsembler, GestureTransformer

#### Set parameters

In [27]:
is_frame_based = True
ms_per_frame_original = 120
ms_per_frame_interpolated = 50
use_rescaling_to_true_length = False
use_interpolation = False
is_verbose = True
ms_per_frame_interpolated

50

#### Load, interpolate and assemble data

In [16]:
de = DataEnsembler(ms_per_frame=ms_per_frame_original)
de.investigate_available_datafiles(data_dir='data/gesture/', is_frame_based = is_frame_based)
de.load_data()

In [17]:
if use_rescaling_to_true_length:
    de.rescale_data_frames(time_of_first_frame = 'avg', verbose=is_verbose)

if use_interpolation:
    if is_frame_based:
        de.interpolate_and_convert_framebased_labels(new_frmlen=ms_per_frame_interpolated, verbose=is_verbose)
    else:
        de.interpolate_data_frames(frmlen=ms_per_frame_interpolated, verbose=is_verbose)

Current Index: 0
Calling DataFrameInterpolator.scaleDataFrame method with current length 66290.0 and actual length 68677
Using 'avg' to calculate new time of first frame: 0.0 ==> 106
Current Index: 1
Calling DataFrameInterpolator.scaleDataFrame method with current length 60562.0 and actual length 63117
Using 'avg' to calculate new time of first frame: 0.0 ==> 112
Current Index: 2
Calling DataFrameInterpolator.scaleDataFrame method with current length 63629.0 and actual length 64048
Using 'avg' to calculate new time of first frame: 0.0 ==> 123
Current Index: 3
Calling DataFrameInterpolator.scaleDataFrame method with current length 69924.0 and actual length 70576
Using 'avg' to calculate new time of first frame: 0.0 ==> 104
Current Index: 4
Calling DataFrameInterpolator.scaleDataFrame method with current length 62303 and actual length 62915
Using 'avg' to calculate new time of first frame: 0 ==> 118
Current Index: 5
Calling DataFrameInterpolator.scaleDataFrame method with current length 

In [18]:
if is_frame_based:
    de.assemble_data()
else:
    de.assemble_data(tolerance_range = 600, max_error = 550)
    
de.display_information()

i: 0 	shape X: (1356, 18, 16) 	shape y: (1356,) 	count: 144
i: 1 	shape X: (1244, 18, 16) 	shape y: (1244,) 	count: 168
i: 2 	shape X: (1263, 18, 16) 	shape y: (1263,) 	count: 192
i: 3 	shape X: (1394, 18, 16) 	shape y: (1394,) 	count: 264
i: 4 	shape X: (1240, 18, 16) 	shape y: (1240,) 	count: 156
i: 5 	shape X: (1267, 18, 16) 	shape y: (1267,) 	count: 180
i: 6 	shape X: (1317, 18, 16) 	shape y: (1317,) 	count: 156
i: 7 	shape X: (1251, 18, 16) 	shape y: (1251,) 	count: 120
i: 8 	shape X: (1273, 18, 16) 	shape y: (1273,) 	count: 132
i: 9 	shape X: (1264, 18, 16) 	shape y: (1264,) 	count: 144
i: 10 	shape X: (1362, 18, 16) 	shape y: (1362,) 	count: 120
i: 11 	shape X: (1217, 18, 16) 	shape y: (1217,) 	count: 96
i: 12 	shape X: (1278, 18, 16) 	shape y: (1278,) 	count: 132
i: 13 	shape X: (1232, 18, 16) 	shape y: (1232,) 	count: 0
i: 14 	shape X: (1203, 18, 16) 	shape y: (1203,) 	count: 12
i: 15 	shape X: (1242, 18, 16) 	shape y: (1242,) 	count: 12
i: 16 	shape X: (1286, 18, 16) 	shape y

In [26]:
de.combined_data_files_df

,filename_features,filename_labels
0,features_flip_c_01_120.csv,labels_flip_c_01.csv
1,features_flip_c_02_120.csv,labels_flip_c_02.csv
2,features_flip_c_03_120.csv,labels_flip_c_03.csv
3,features_flip_l_01_120.csv,labels_flip_l_01.csv
4,features_flip_l_02_120.csv,labels_flip_l_02.csv
5,features_flip_l_03_120.csv,labels_flip_l_03.csv
6,features_flip_p_01_120.csv,labels_flip_p_01.csv
7,features_land_c_01_120.csv,labels_land_c_01.csv
8,features_land_c_02_120.csv,labels_land_c_02.csv
9,features_land_c_03_120.csv,labels_land_c_03.csv


In [19]:
feature_names = list(de.feature_names)
feat_dict = {feature:feature_names.index(feature) for feature in feature_names}
feat_dict

{'leftElbow_x': 0,
 'leftElbow_y': 1,
 'leftHip_x': 2,
 'leftHip_y': 3,
 'leftShoulder_x': 4,
 'leftShoulder_y': 5,
 'leftWrist_x': 6,
 'leftWrist_y': 7,
 'rightElbow_x': 8,
 'rightElbow_y': 9,
 'rightHip_x': 10,
 'rightHip_y': 11,
 'rightShoulder_x': 12,
 'rightShoulder_y': 13,
 'rightWrist_x': 14,
 'rightWrist_y': 15}

In [ ]:
def plot_charts(de, file_id, tail=5, head=5, plots_per_row=30, body_part='leftWrist', is_frame_based = True):

    global movements
    
    kx = body_part + '_x'
    ky = body_part + '_y'

    lgen = de.LabelGenerators[file_id]
    y = lgen.y
    n = len(lgen.y)

    gt = GestureTransformer(byrow=True, feature_names=feature_names)
    X = gt.transform(lgen.X)
    s = lgen.X.shape[1]

    d = lgen.labeled_data
    dims = d[feature_names].shape
    D = d[feature_names].values.reshape(1,dims[0],dims[1])
    D = gt.transform(D).reshape(dims[0],dims[1])
    Dx = D[:,feat_dict[kx]]
    Dy = D[:,feat_dict[ky]]    
    
    for j in range(n // plots_per_row + 1):
        
        plt.figure(figsize=(2 * plots_per_row, 2))
        
        for i in range (plots_per_row):
            plt.subplot(1, plots_per_row, i + 1)
            index = j*plots_per_row + i

            # specify axes range
            axes = plt.gca()
            axes.set_xlim([-2,2])
            axes.set_ylim([-2,2])
            axes.get_yaxis().set_visible(False)
            axes.get_xaxis().set_visible(False)

            if index < n:
                title_suffix = ''
                if (int(y[index])>0):
                    axes.set_facecolor((0.95, 0.95, 0.95))
                    title_suffix = ' | ' + movements[int(y[index])]

                # movement
                l1, u1 = max(0,index - s), index
                plt.plot(Dx[l1:u1],Dy[l1:u1],color = "xkcd:baby blue")

                # past
                l2, u2 = max(0,index - s - tail), max(0,index-s)
                plt.plot(Dx[l2:u2+1],Dy[l2:u2+1],linestyle = "--",color = "green")

                # future
                l3, u3 = index, min(index + head,dims[0])
                plt.plot(Dx[l3:u3],Dy[l3:u3],linestyle = "--",color = "red")
                
                # current frame
                plt.plot(Dx[u1-1],Dy[u1-1],color = "xkcd:baby blue",marker = "o")
                
                if is_frame_based:
                    plt.title(str(index) + title_suffix)
                else:
                    plt.title(str(index) + "/" + str(round(lgen.sequence_end_time[index]/1000,2)) + " l=" + str(int(y[index])))
                

        plt.show()
        plt.close()    

Available body parts:
```
leftElbow
leftHip
leftShoulder
leftWrist
rightElbow
rightHip
rightShoulder
rightWrist
```

In [ ]:
de.combined_data_files_df

**Legend:**<br>
<span style="color:blue">gesture</span><br>
<span style="color:red">frames *after* gesture</span><br>
<span style="color:green">frames *before* gesture</span>

In [ ]:
movements = {0: 'not detected',
             1: 'takeoff',
             2: 'move',
             3: 'flip',
             4: 'left',
             5: 'right',
             6: 'land'}

In [ ]:
file_id = 33
plot_charts(de, file_id=file_id, tail=5, head=5, plots_per_row=30, body_part='leftWrist', is_frame_based = is_frame_based)

Display the movement endpoints (in milliseconds) of all non-zero labeled gestures:

In [ ]:
de.LabelGenerators[file_id].label_df.join(de.LabelGenerators[file_id].label_info)

Display the indices of movement ends of all non-zero labeled gestures:

In [ ]:
lgen = de.LabelGenerators[file_id]
y = lgen.y

gt = GestureTransformer(byrow=True, feature_names=feature_names)
X = gt.transform(lgen.X)

gt = GestureTransformer(feature_names)
P = gt.transform(de.X)

key = 'leftWrist'
kx = key + '_x'
ky = key + '_y'

# get the label which is used in the current file
lab = y[y>0][0]

this_set = True
if not this_set: #use all the data
    movementData = P[np.isclose(de.y,lab),:,:]
else:
    movementData = X[np.isclose(y,lab),:,:]

print(movementData.shape)
movementData = movementData[:,:,[feat_dict[kx],feat_dict[ky]]]
avgMovement = movementData.mean(axis = 0)
avgMovement